In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
# from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *

In [5]:
#function to create api connection to google sheets
def connect_to_gs(service_account_key):
    scopes = ['https://www.googleapis.com/auth/spreadsheets']
    credentials = service_account.Credentials.from_service_account_info(service_account_key, scopes=scopes)
    gs_connection = gspread.authorize(credentials)
    return gs_connection


#create connection
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [3]:
#function to create/update gameweek_results table

In [4]:
#function to create/update player table

In [5]:
#function to create/update drinks table

# def submit_nomination(your_name, nominee, current_week):

#     return None

In [6]:
#function to create/update streaks table

In [7]:
#function to retrieve current game week number

In [4]:
#function to write data to google sheets
def write_google_sheets_data(gc, df, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        df_values = df.values.tolist()
        gs.values_append(sheet_name, {'valueInputOption': 'RAW'}, {
                    'values': df_values})

        return None

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"Error: Worksheet not found, please create a new tab named:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
     

In [6]:
#function to fetch data from google sheets
def fetch_google_sheets_data(gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
    
#fetch data


In [87]:
managers_temp = ['Liam Smorfitt', 'Alex Wietzorrek', 'Hethe Brinkman']

In [88]:
red_cards =['Liam Smorfitt', 'Alex Wietzorrek',]

In [9]:
#function to return season metrics
def create_metrics(df):
    
    # Sort the DataFrame in descending order of points, then descending order of total_points then alphabetically
    df_sorted = df.sort_values(['event', 'points', 'total_points', 'player_name'], ascending=[True, False, False, True])

    # Group the DataFrame by event and get the first place finisher for each event
    first_place_df = df_sorted.groupby('event').head(1)

    # Count the occurrences of each entry in the filtered DataFrame
    first_place_player_counts = first_place_df['player_name'].value_counts()

    # Find the player with the most 1st place finishes
    most_1st_place_player = first_place_player_counts.idxmax()
    most_1st_place_count = first_place_player_counts.max()


    # Sort the DataFrame in descending order of points, then descending order of total_points then alphabetically
    df_sorted = df.sort_values(['event', 'points', 'total_points', 'player_name'], ascending=[True, False, False, False])

    # Group the DataFrame by event and get the last place finisher for each event
    last_place_df = df_sorted.groupby('event').tail(1)

    # Count the occurrences of each player in the last place DataFrame
    last_place_player_counts = last_place_df['player_name'].value_counts()

    # Find the player with the most last place finishes
    most_last_place_player = last_place_player_counts.idxmax()
    most_last_place_count = last_place_player_counts.max()


    # Group the DataFrame by player_name and calculate the sum of event_transfer_cost for each player
    player_transfer_cost = df.groupby('player_name')['event_transfers_cost'].sum()

    # Find the player with the highest total event_transfer_cost
    player_with_highest_cost = player_transfer_cost.idxmax()
    player_with_highest_cost_count = player_transfer_cost.max()/4

    # Group the DataFrame by player_name and calculate the sum of points_on_bench for each player
    player_points_on_bench = df.groupby('player_name')['points_on_bench'].sum()

    # Find the player with the highest total points_on_bench
    player_with_highest_points_on_bench = player_points_on_bench.idxmax()
    player_with_highest_points_on_bench_count = player_points_on_bench.max()


    # Find the row with the lowest points across all events
    min_score_row = df.loc[df['points'].idxmin()]

    # Extract the player_name and event for the row with the lowest score
    lowest_score_player_name = min_score_row['player_name']
    lowest_score_event = min_score_row['event']
    lowest_score_points = min_score_row['points']


    return most_1st_place_player, most_1st_place_count, most_last_place_player, most_last_place_count, player_with_highest_cost, player_with_highest_cost_count, player_with_highest_points_on_bench, player_with_highest_points_on_bench_count,lowest_score_player_name,lowest_score_event, lowest_score_points

In [ ]:
#writes nominations to googlesheets

def record_nomination(df, gc, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
    

In [100]:
def fetch_max_gw(gc, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        # Convert the first column to numeric
        df[headers[0]] = pd.to_numeric(df[headers[0]])

        # Get the maximum value of the first column
        max_value = df[headers[0]].max()

        return max_value

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None


In [114]:
np.random.randint(1,5)

4

In [105]:
maxgw = fetch_max_gw(gs_connection, 'Sheet11', google_sheet_key,)

In [106]:
maxgw

nan

In [17]:
#fetch nominations data 
df = google_sheets_data(gs_connection, nominations_table, google_sheet_key, [])

In [43]:
def submit_drink(gc, df, sheet_key, nominee):
    try:
        filtered_df = df[(drinks_df["drinker_name"] == nominee) & (df["nomination_completed_date"] == "Not Completed")]
        last_record_index = filtered_df.index[-1]
        df.at[last_record_index, "nomination_completed_date"] = (datetime.now() + timedelta(hours=2)).strftime("%d/%m/%y %H:%M:%S")
    except IndexError as e:
        r = "You dont have any outstanding drinks"
        return r

    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet("drinks")

        set_with_dataframe(tab, df)

        return None

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"Error: Worksheet not found, please create a new tab named:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None


In [ ]:
#uno reverse function, finds last incompleted record from drinks table and switches the nominator and nominee name
def uno_reverse(gc, df, sheet_key, nominee):
    try:
        filtered_df = df[(drinks_df["drinker_name"] == nominee) & (df["nomination_completed_date"] == "Not Completed")]
        last_record_index = filtered_df.index[-1]
        if 
        df.at[last_record_index, "drinker_name"] = filtered_df.nominator[last_record_index]
    except IndexError as e:
        r = "You dont have any outstanding drinks"
        return r

    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet("drinks")

        set_with_dataframe(tab, df)

        return None

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"Error: Worksheet not found, please create a new tab named:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None


In [42]:
def categories(df):
    df['number_of_drinks'] = 1
    df = df.iloc[:, [2,5,6,7]]
    # Define the conditions for the new column
    condition1 = (df['nomination_completed_date'] < df['nomination_deadline_date'])
    condition2 = (df['nomination_completed_date'] > df['nomination_deadline_date'])
    condition3 = (df['nomination_completed_date'] == 'Not Completed')

    # Apply the conditions and create the new column
    df['categories'] = 'Default Value'  # Set a default value if no conditions are met
    df.loc[condition1, 'categories'] = 'Completed'
    df.loc[condition2, 'categories'] = 'Late'
    df.loc[condition3, 'categories'] = 'Outstanding'

    return df


In [4]:
drinks = fetch_google_sheets_data(gs_connection, drinks_table, google_sheet_key, ['event','drink_size', 'start_time', 'end_time'])


In [59]:
drinks

,event,nominator_name,drinker_name,drink_type,nomination_created_date,nomination_deadline_date,nomination_completed_date,drink_size,start_time,end_time
0,36,Ryan Shacks,Divyam Dixit,nomination,16/07/23 22:15:58,21/07/23 22:02,22/07/23 22:18,500.0,11.23,12.49
1,37,Ryan Shacks,Liam Smorfitt,nomination,22/07/23 22:18:48,24/07/23 22:00,27/07/23 12:18,500.0,7.75,12.50
2,37,Ryan Shacks,Connor McDonald,nomination,22/07/23 22:18:48,29/07/23 22:00,29/07/23 21:55,330.0,3.68,8.32
3,37,Peter Wertz,Ryan Shacks,uno reverse,22/07/23 22:18:48,29/07/23 22:00,23/07/23 12:18,330.0,6.90,10.11
4,38,Ryan Shacks,Liam Smorfitt,nomination,24/07/23 18:00:33,31/07/23 18:00,29/07/23 21:51,330.0,4.61,15.22
5,38,Ryan Shacks,Jason Perry,nomination,24/07/23 18:00:33,31/07/23 18:00,29/07/23 21:50,330.0,1.20,5.53
6,38,Ryan Shacks,Devon Jansen,nomination,29/07/23 21:59:05,05/08/23 21:00,Not Completed,NaN,NaN,NaN
7,38,Ryan Shacks,Liam Smorfitt,nomination,29/07/23 21:59:05,05/08/23 21:00,30/07/23 17:58,500.0,NaN,NaN
8,38,Ryan Shacks,Connor McDonald,nomination,29/07/23 21:59:05,05/08/23 21:00,29/07/23 22:04,330.0,5.64,8.93
9,38,Ryan Shacks,Jason Perry,nomination,30/07/23 18:06:58,06/08/23 18:00,Not Completed,NaN,NaN,NaN


In [68]:
drinks_lap_times = drinks.iloc[:,[0,2,5,6,7,8,9]].dropna()
drinks_lap_times = drinks_lap_times[drinks_lap_times['nomination_completed_date'] < drinks_lap_times['nomination_deadline_date']]

In [69]:
drinks_lap_times

,event,drinker_name,nomination_deadline_date,nomination_completed_date,drink_size,start_time,end_time
2,37,Connor McDonald,29/07/23 22:00,29/07/23 21:55,330.0,3.68,8.32
3,37,Ryan Shacks,29/07/23 22:00,23/07/23 12:18,330.0,6.90,10.11
4,38,Liam Smorfitt,31/07/23 18:00,29/07/23 21:51,330.0,4.61,15.22
5,38,Jason Perry,31/07/23 18:00,29/07/23 21:50,330.0,1.20,5.53
11,38,Hethe Brinkman,06/08/23 18:00,05/08/23 8:51,440.0,1.02,4.12


In [70]:
drinks_lap_times['completion_time'] = np.round((drinks_lap_times['end_time'] - drinks_lap_times['start_time'])*(330/drinks['drink_size']),3)

In [71]:
drinks_lap_times = drinks_lap_times.sort_values(by=['completion_time'])
fastest_time = drinks_lap_times['completion_time'].min()

In [72]:
drinks_lap_times['gap'] =  drinks_lap_times['completion_time'] - fastest_time
drinks_lap_times ['gap'] = drinks_lap_times['gap'].apply(lambda x: f"+{x:.3f}" if pd.notna(x) and x != 0 else '')

In [73]:
drinks_lap_times

,event,drinker_name,nomination_deadline_date,nomination_completed_date,drink_size,start_time,end_time,completion_time,gap
11,38,Hethe Brinkman,06/08/23 18:00,05/08/23 8:51,440.0,1.02,4.12,2.325,
3,37,Ryan Shacks,29/07/23 22:00,23/07/23 12:18,330.0,6.90,10.11,3.210,+0.885
5,38,Jason Perry,31/07/23 18:00,29/07/23 21:50,330.0,1.20,5.53,4.330,+2.005
2,37,Connor McDonald,29/07/23 22:00,29/07/23 21:55,330.0,3.68,8.32,4.640,+2.315
4,38,Liam Smorfitt,31/07/23 18:00,29/07/23 21:51,330.0,4.61,15.22,10.610,+8.285


In [74]:
display_times = drinks_lap_times.iloc[:, [0,1,7,8]]

In [75]:
display_times

,event,drinker_name,completion_time,gap
11,38,Hethe Brinkman,2.325,
3,37,Ryan Shacks,3.210,+0.885
5,38,Jason Perry,4.330,+2.005
2,37,Connor McDonald,4.640,+2.315
4,38,Liam Smorfitt,10.610,+8.285


In [93]:
def compute_laps(df): 
    drinks_lap_times = df.iloc[:,[0,2,5,6,7,8,9]].dropna()
    drinks_lap_times = drinks_lap_times[drinks_lap_times['nomination_completed_date'] < drinks_lap_times['nomination_deadline_date']]   
    drinks_lap_times['completion_time'] = np.round((drinks_lap_times['end_time'] - drinks_lap_times['start_time'])*(330/drinks['drink_size']),3)
    drinks_lap_times = drinks_lap_times.sort_values(by=['completion_time'])
    fastest_time = drinks_lap_times['completion_time'].min()
    drinks_lap_times['gap'] =  drinks_lap_times['completion_time'] - fastest_time
    drinks_lap_times ['gap'] = drinks_lap_times['gap'].apply(lambda x: f"+{x:.3f}" if pd.notna(x) and x != 0 else '')   
    display_times = drinks_lap_times.iloc[:, [0,1,7,8]]
    display_times.index = np.arange(1, len(display_times) + 1)


    display_times.rename(
        columns={"event": "Game Week", "drinker_name": "Driver", "completion_time": 'Lap Time', "gap": "Gap"}, inplace=True
    )
    return display_times
    

In [95]:
def compute_laps(df):
    # Filter rows with relevant columns and non-null values
    drinks_lap_times = df[['event', 'drinker_name', 'nomination_completed_date', 'nomination_deadline_date', 
                          'start_time', 'end_time', 'drink_size']].dropna()

    # Filter rows with valid nomination completion date
    drinks_lap_times = drinks_lap_times[drinks_lap_times['nomination_completed_date'] < drinks_lap_times['nomination_deadline_date']]

    # Compute the lap times
    drinks_lap_times['completion_time'] = np.round((drinks_lap_times['end_time'] - drinks_lap_times['start_time']) * (330 / drinks_lap_times['drink_size']), 3)

    # Sort by completion_time
    drinks_lap_times = drinks_lap_times.sort_values(by='completion_time')

    # Compute the gap from the fastest time
    fastest_time = drinks_lap_times['completion_time'].min()
    drinks_lap_times['gap'] = drinks_lap_times['completion_time'] - fastest_time
    drinks_lap_times['gap'] = drinks_lap_times['gap'].apply(lambda x: f"+{x:.3f}" if pd.notna(x) and x != 0 else '')

    # Select and format the relevant columns
    display_times = drinks_lap_times[['event', 'drinker_name', 'completion_time', 'gap']]
    display_times.index = np.arange(1, len(display_times) + 1)

    # Rename the columns
    display_times.rename(
        columns={"event": "Game Week", "drinker_name": "Driver", "completion_time": "Lap Time", "gap": "Gap"},
        inplace=True
    )
    return display_times

In [28]:
uno_data = fetch_google_sheets_data(gs_connection, managers_table, google_sheet_key, [])
uno_data_display = uno_data.iloc[:, [1, 3]].sort_values(["uno_reverse", "player_name"], ascending=(False, True))
uno_data_display.rename(columns={"player_name": "Name", "uno_reverse": "Has Uno Reverse"}, inplace=True)
uno_data_display.index = np.arange(1, len(uno_data) + 1)

In [29]:
uno_data

,entry,player_name,event_joined,uno_reverse
0,1742534,Niklas Wietzorrek,1,Yes
1,1930701,Liam Smorfitt,1,Yes
2,2208836,Divyam Dixit,1,Yes
3,552345,Marco Gouveia,1,Yes
4,578302,Ryan Shacks,1,Yes
5,2123591,Peter Wertz,1,Yes
6,1002132,Devon Hodgson,1,Yes
7,534613,Connor McDonald,1,No
8,2032628,Hethe Brinkman,1,Yes
9,3648627,Jason Perry,1,No


In [41]:
nominee = 'Connor McDonald'

# Check if the nominee has already used their uno reverse card this season
try:
    filtered_uno_data = uno_data[(uno_data["player_name"] == nominee)]
    uno_index = uno_data[(uno_data["player_name"] == nominee)].index[0]

    if filtered_uno_data.iloc[0, 3] == 'No':
        raise Exception("You have already used your uno reverse card this season")
    else: 
        # Use a single equal sign (=) for assignment
        uno_data.at[uno_index, 'uno_reverse'] = 'No'
except Exception as e:
    print(e)





You have already used your uno reverse card this season


In [35]:
filtered_uno_data = uno_data[(uno_data["player_name"] == nominee)]
filtered_uno_data 

,entry,player_name,event_joined,uno_reverse
7,534613,Connor McDonald,1,No


In [36]:
uno_index = uno_data[(uno_data["player_name"] == nominee)].index[0]
uno_index

7

In [40]:
filtered_uno_data.iloc[0, 3]

'No'

In [39]:
filtered_uno_data

,entry,player_name,event_joined,uno_reverse
7,534613,Connor McDonald,1,No


In [31]:
uno_data

,entry,player_name,event_joined,uno_reverse
0,1742534,Niklas Wietzorrek,1,Yes
1,1930701,Liam Smorfitt,1,Yes
2,2208836,Divyam Dixit,1,Yes
3,552345,Marco Gouveia,1,Yes
4,578302,Ryan Shacks,1,Yes
5,2123591,Peter Wertz,1,Yes
6,1002132,Devon Hodgson,1,Yes
7,534613,Connor McDonald,1,No
8,2032628,Hethe Brinkman,1,Yes
9,3648627,Jason Perry,1,No


In [14]:
filtered_df

,Name,Has Uno Reverse
1,Alex Wietzorrek,Yes


In [18]:
if filtered_df[filtered_df.columns[1]].values == 'No':
    print(1)